In [1]:
import wmfdata as wmf

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [2]:
num_art = """
select
    database() as wiki,
    ss_good_articles as articles
from site_stats;
"""

# Wikipedia articles

In [3]:
wps = wmf.utils.list_wikis(["wikipedia"])

In [4]:
art = wmf.mariadb.multirun(num_art, wikis = wps)

aawiki completed in 0 s
abwiki completed in 0 s
acewiki completed in 0 s
adywiki completed in 0 s
afwiki completed in 0 s
akwiki completed in 0 s
alswiki completed in 0 s
amwiki completed in 0 s
angwiki completed in 0 s
anwiki completed in 0 s
arcwiki completed in 0 s
arwiki completed in 0 s
arzwiki completed in 0 s
astwiki completed in 0 s
aswiki completed in 0 s
atjwiki completed in 0 s
avwiki completed in 0 s
aywiki completed in 0 s
azbwiki completed in 0 s
azwiki completed in 0 s
barwiki completed in 0 s
bat_smgwiki completed in 0 s
bawiki completed in 0 s
bclwiki completed in 0 s
be_x_oldwiki completed in 0 s
bewiki completed in 0 s
bgwiki completed in 0 s
bhwiki completed in 0 s
biwiki completed in 0 s
bjnwiki completed in 0 s
bmwiki completed in 0 s
bnwiki completed in 0 s
bowiki completed in 0 s
bpywiki completed in 0 s
brwiki completed in 0 s
bswiki completed in 0 s
bugwiki completed in 0 s
bxrwiki completed in 0 s
cawiki completed in 0 s
cbk_zamwiki completed in 0 s
cdowiki c

In [17]:
art.head()

,wiki,articles
0,aawiki,1
1,abwiki,3408
2,acewiki,7206
3,adywiki,408
4,afwiki,49461


In [5]:
art["articles"].sum()

47902207

# Wiktionary entries

In [6]:
wts = wmf.utils.list_wikis(["wiktionary"])

In [7]:
ent = wmf.mariadb.multirun(num_art, wikis = wts)

aawiktionary completed in 0 s
abwiktionary completed in 0 s
afwiktionary completed in 0 s
akwiktionary completed in 0 s
alswiktionary completed in 0 s
amwiktionary completed in 0 s
angwiktionary completed in 0 s
anwiktionary completed in 0 s
arwiktionary completed in 0 s
astwiktionary completed in 0 s
aswiktionary completed in 0 s
avwiktionary completed in 0 s
aywiktionary completed in 0 s
azwiktionary completed in 0 s
bewiktionary completed in 0 s
bgwiktionary completed in 0 s
bhwiktionary completed in 0 s
biwiktionary completed in 0 s
bmwiktionary completed in 0 s
bnwiktionary completed in 0 s
bowiktionary completed in 0 s
brwiktionary completed in 0 s
bswiktionary completed in 0 s
cawiktionary completed in 0 s
chrwiktionary completed in 0 s
chwiktionary completed in 0 s
cowiktionary completed in 0 s
crwiktionary completed in 0 s
csbwiktionary completed in 0 s
cswiktionary completed in 0 s
cywiktionary completed in 0 s
dawiktionary completed in 0 s
dewiktionary completed in 0 s
dvwik

In [18]:
ent.head()

,wiki,articles
0,aawiktionary,0
1,abwiktionary,0
2,afwiktionary,20342
3,akwiktionary,0
4,alswiktionary,0


In [8]:
ent["articles"].sum()

28067920

# Wikidata items

In [9]:
wmf.mariadb.run("select ss_good_articles from wikidatawiki.site_stats")

,ss_good_articles
0,47218270


# Commons files

In [11]:
licenses_q = """
select license, count(*) as files
from
(select
  case
    when cats like '%CC-BY-NC-ND%' then 'CC-BY-NC-ND'
    when cats like '%CC-BY-NC-SA%' then 'CC-BY-NC-SA'
    when cats like '%CC-BY-ND%' then 'CC-BY-ND'
    when cats like '%CC-BY-SA%' then 'CC-BY-SA'
    when cats like '%CC-BY-NC%' then 'CC-BY-NC'
    when cats like '%CC-BY%' then 'CC-BY'
    when cats like '%CC-SA%' then 'CC-SA'
    when cats like '%CC-Zero%' then 'CC-0'
    when cats like '%CC-PD%' then 'CC-PD'
    else 'Other CC'
  end as license
  from
  (select cl_from, group_concat(cl_to) as cats
    from commonswiki.categorylinks
    inner join
      (select cat_title from commonswiki.category where
        cat_title like 'CC-%' and
        cat_title not like '%aircraft%' and
        cat_title not regexp 'CC-[[:upper:][:digit:]]{3}'
      ) cc_cats
    on cat_title = cl_to
    where cl_type = "file"
    group by cl_from
  ) cc_files
) licenses
group by license;
"""

licenses = wmf.mariadb.run(licenses_q)

In [12]:
licenses.sort_values("license")

,license,files
0,CC-0,2556807
1,CC-BY,6649957
2,CC-BY-NC,6705
3,CC-BY-NC-SA,231
4,CC-BY-ND,15
5,CC-BY-SA,26282103
6,CC-PD,4269282
7,CC-SA,2799
8,Other CC,1


In [13]:
licenses["files"].sum()

39767900

In [ ]:
media_types_q = """
select img_media_type, count(*) as files
from
(select distinct cl_from
  from commonswiki.categorylinks
  inner join
    (select cat_title from commonswiki.category where
      cat_title like 'CC-%' and
      cat_title not like '%aircraft%' and
      cat_title not regexp 'CC-[[:upper:][:digit:]]{3}'
    ) cc_cats
  on cat_title = cl_to
  where cl_type = "file"
) cc_files
inner join commonswiki.page on cl_from = page_id
inner join commonswiki.image on page_title = img_name
group by img_media_type;
"""

media_types = wmf.mariadb.run(media_types_q)

In [15]:
media_types

,img_media_type,files
0,BITMAP,37784741
1,DRAWING,819140
2,AUDIO,799534
3,VIDEO,103970
4,MULTIMEDIA,4
5,OFFICE,260210
6,3D,281


In [16]:
media_types["files"].sum()

39767880

## Total Commons files

In [10]:
wmf.mariadb.run("select ss_images from commonswiki.site_stats")

,ss_images
0,46474917
